# Contour Overlay 

## Imports 
* _numpy_ for array math
* _astropy.io_ for reading and writing FITS cubes and images
* _matplotlib.pyplot_ for plotting spectra and images.

In [ ]:
from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
import numpy as np

import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## Read in data cube
Read in NIRSpec IFU data cube from Box.   This particular example is a simulated quasar + host galaxy.

In [ ]:
BoxPath='https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/cube_fitting/Q3D_20200407/'
fname='Q3D_NRS_491_s3d.fits'
filename=BoxPath+fname
hdul1=fits.open(filename)
hdul1.info()

with fits.open(filename, memmap=False) as hdulist:
    sci = hdulist["SCI"].data


## Sum over spectral axis

This could be any derivative data product of the same shape as the spatial dimensions of the cube (emission line flux, equivalent width, velocity map, etc...). Here we are doing one of the simplest operations - a sum, for illustrative purposes

In [ ]:
cube_sum=np.sum(sci, axis=0)

## Generate contours
User-specified contour levels in specified units. Custom colors are important, but could be added at a later stage if technically complicated.  Logarithmic and linear spacing options to auto-generate contours would be nice.

In [ ]:
min_level=np.min(cube_sum)
max_level=np.max(cube_sum)
level_colors=['blue', 'purple','red', 'magenta']
contour_levels=[max_level/10000., max_level/1000., max_level/100., max_level/10.]

## Display image and overlay contours

In [ ]:
ax = plt.subplots()[1]

#Image normalization
norm = ImageNormalize(stretch=SqrtStretch())

#Display image
ax.imshow(cube_sum, cmap='gray', origin='lower', norm=norm)

#Overlay contours
ax.contour(cube_sum, levels=contour_levels, colors=level_colors)
